# OmniSci - Ibis - Holoviews - hvPlot

![](https://avatars1.githubusercontent.com/u/7553829?s=200&v=4)    ![](https://avatars1.githubusercontent.com/u/34879953?s=200&v=4) 

### Bio

Tony Fast is a Developer Advocate at Quansight with a passion for literate programing and a contributor to Jupyter and HoloViz.

Kim Pevey is a Software Engineer at Quansight with an interest for analyzing big data with visualization tools in python. She is also a contributor to HoloViz.

### Abstract
This notebook demonstrates visualizations of OmniSci Database data using   
the Holoviz tools Holoviews and hvPlot. The connection between the OmniSci  
Database and Holoviz is an OmniSci backend in Ibis

In [ ]:
    __import__("graphviz").Source("""graph {layout=circo 
    {{intake -- {ibis -- pandas}} -- {holoviews}}--hvplot geoviews--{holoviews hvplot}[style=dotted]}
    """)

### `holoviews` - a high level plotting interface

### `hvplot` - a plotting interface for python datatypes

> #### We'll demonstrate these tools by plotting voting data from 2008. Let's get started!

In [ ]:
import ibis
import holoviews as hv
from holoviews import opts
import numpy as np
import hvplot.ibis

from ibis.backends import omniscidb as ibis_omniscidb

hv.extension('bokeh')

### establish the connection to the database

In [ ]:
client = ibis_omniscidb.connect(
    user="mapd", 
    password="HyperInteractive", 
    host="metis.mapd.com", 
    database="mapd", 
    port=443, 
    protocol='https'
)

### get the table

In [ ]:
votes = client.table('demo_vote_clean')
votes

In [ ]:
votes.head().execute()

> #### we'll need to convert the percentage of votes for each candidate into a concrete number of votes using `ibis`


### convert percentages to total votes

In [ ]:
votes = votes.mutate(
        democrat=votes.Population.mul(votes.Obama),
        republican=votes.Population.mul(votes.Romney),
    )

### aggregate votes by state (StateAbb)

In [ ]:
voted = votes.groupby('StateAbb').aggregate(democrat=votes.democrat.sum(), republican=votes.republican.sum())

We can pass our new table directly to the Bar Chart constructor in Holoviews.   
Since we haven't provided and specifics of what we'd like to see displayed,   
Holoviews takes a guess at the dimensions for the plot. 

In [ ]:
hv_bar = hv.Bars(voted).opts(show_grid=True, width=900, height=500)
hv_bar

Let's look at what Holoviews came up with.

In [ ]:
print(f'kdims: {hv_bar.kdims}')
print(f'vdims: {hv_bar.vdims}')

### overlayed `holoviews` plots

> #### Holoviews has chosen the correct kdims, but the Bar chart can only visualize  one vdim at a time. The overlay functionality will allow us to construct two Bar charts, one each for `democrat` and `republican`, and layer them on top of one another.

In [ ]:
(
    hv.Bars(voted, 
            kdims=['StateAbb'], 
            vdims=['democrat'])
    # * is overlay that places plots on the same axes
    * hv.Bars(voted, 
              kdims=['StateAbb'], 
              vdims=['republican'])
).opts(
    xlabel='State Abbreviation', 
    ylabel='Number of votes', 
    active_tools=['wheel_zoom','pan'],
    title='Number of Democrat (Blue) and Republican (Red) Votes per State', 
    show_grid=True, 
    width=900, 
    height=500
)

### overlayed `hvplot`

Let's create the same view using hvplot.

##### overlayed axes

In [ ]:
(
    voted.hvplot.bar('StateAbb', 'democrat')
    * voted.hvplot.bar('StateAbb', 'republican')
).opts(width=1000)

##### side by side axes

In [ ]:
voted = voted.head(10)
(
    (voted.hvplot.bar('StateAbb', 'democrat'))
    + voted.hvplot.bar('StateAbb', 'republican')
)

We can create the same plot as the lower level Holoviews library, but  
hvplot doesn't require you to explicitly set the dimensions. Instead, it    
allows users to ask for the bits of data that they'd like to see plotted.    
Hvplot sorts out the details for you.

### Onto a really complicated example.
### hold on to your hats.